In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

In [ ]:
feature_x_sample = pd.read_table(
    '../output/max_tpm__gene_x_sample.tsv',
    index_col=0,
)

In [ ]:
target_x_sample = pd.read_table(
    '../data/target_x_sample.tsv',
    index_col=0,
)

target_x_sample

In [ ]:
np.random.seed(20121020)

from statsmodels.sandbox.stats.multicomp import multipletests

n_permutation = 100

for target_x_sample__index, target in target_x_sample.iterrows():
    
    target = target[target!=-1]
    
    feature_x_sample_ = feature_x_sample[target.index]
    
    feature_x_target = feature_x_sample_.groupby(
        by=target,
        axis=1,
    ).mean()
    
    feature_score = ccal.compute_log_ratio_between_2_1d_arrays(
        feature_x_target.iloc[:, 0] + 1, 
        feature_x_target.iloc[:, 1] + 1,
        log_base='2',
    ) 
    
    feature_x_permutation = np.full(
        (
            feature_x_sample.shape[0],
            n_permutation,
        ),
        np.nan,
    )

    for i in range(n_permutation):

        np.random.shuffle(target)

        feature_x_target = feature_x_sample_.groupby(
            by=target,
            axis=1,
        ).mean()

        feature_x_permutation[:, i] = ccal.compute_log_ratio_between_2_1d_arrays(
            feature_x_target.iloc[:, 0] + 1, 
            feature_x_target.iloc[:, 1] + 1,
            log_base='2',
        ) 

    p_values = np.full(
        feature_score.size,
        np.nan,
    )

    for i, score in enumerate(feature_score):

        random_scores = feature_x_permutation[i]

        p_values[i] = min(
            ccal.compute_empirical_p_value(
                score,
                random_scores,
                'less'
            ),
            ccal.compute_empirical_p_value(
                score,
                random_scores,
                'great'
            ),
        )

    feature_p_value = pd.Series(
        p_values,
        index=feature_score.index,
    )


    feature_score_p_value_fdr = pd.concat(
        (
            feature_score,
            feature_p_value,
            pd.Series(
                multipletests(
                    feature_p_value,
                    method='fdr_bh',
                )[1],
                index=feature_p_value.index,
            ),
        ),
        axis=1,
    )

    feature_score_p_value_fdr.columns = (
        'Score',
        'P-Value',
        'FDR',
    )

    feature_score_p_value_fdr.sort_values(
        'P-Value',
        inplace=True,
    )

    print(feature_score_p_value_fdr.head(n=32))
    
    feature_score_p_value_fdr.to_csv(
        '../output/gene/{} Fold Change.tsv'.format(target_x_sample__index),
        sep='\t',
    )

    scores_0 = feature_score_p_value_fdr['Score']

    scores_1 = pd.read_table(
        '../output/gene/{}.tsv'.format(target_x_sample__index),
        index_col=0,
    )['Score']

    scores_0 = scores_0[scores_0.index & scores_1.index].sort_values()

    scores_1 = scores_1[scores_0.index]

    ccal.plot_points(
        ('Scores', ),
        (scores_0, ),
        (scores_1, ),
        texts=(scores_0.index, ),
    )